In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -qU \
    langchain \
    langgraph \
    langchain-huggingface \
    langchain-community \
    faiss-cpu \
    pypdf \
    pymupdf \
    beautifulsoup4 \
    trafilatura \
    datasets \
    sentence-transformers \
    pandas \
    huggingface_hub \
    evaluate \
    scikit-learn \
    bert-score \
    rouge-score \
    flashrank \
    transformers \
    accelerate \
    bitsandbytes \
    langchain-groq \
    torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 142.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 152.5 MB/s eta 0:00:00
  

In [3]:
import os
from google.colab import userdata

# Load API key from Colab secrets
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# Optional: Set LangSmith key for debugging
if userdata.get('LANGCHAIN_API_KEY'):
    os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
else:
    print("⚠️ LANGCHAIN_API_KEY not found in Colab userdata.")

In [4]:
import os
import re
import hashlib
from typing import List

import torch
from datasets import load_dataset
from tqdm import tqdm

# LangChain imports
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ---------------- CONFIG ----------------
# UPDATE THIS PATH to where your PDFs are actually stored
PDF_DATA_PATH = "/content/drive/MyDrive/Agri_pdfs"
KDV_DATASET_ID = "KisanVaani/agriculture-qa-english-only"

VECTORSTORE_PATH = "faiss_index_bge_m3"
EMBEDDING_MODEL = "BAAI/bge-m3"

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100
MAX_KDV_SAMPLES = None        # None = full dataset
USE_CUDA_FOR_EMB = True
# ----------------------------------------

# ---------------- HELPERS ----------------
def normalize_text(s: str) -> str:
    """Clean and normalize text."""
    if not s:
        return ""
    s = str(s).strip()
    s = re.sub(r'\s+', ' ', s)
    return s

def make_key_from_qa(question: str, answer: str) -> str:
    """Unique hash based on Q/A pair."""
    q = normalize_text(question).lower()
    a = normalize_text(answer).lower()
    h = hashlib.sha1(f"{q}||{a}".encode("utf-8")).hexdigest()
    return h

def _resolve_answers_field(answers_field) -> str:
    """Handles both list answers and string answers."""
    if answers_field is None:
        return ""
    if isinstance(answers_field, (list, tuple)):
        for a in answers_field:
            if a:
                s = str(a).strip()
                if s: return s
        return ""
    return str(answers_field).strip()

# ---------------- LOAD KISANVAANI ----------------
def load_kisanvani_dataset(limit: int = None) -> List[Document]:
    print(f"Loading KisanVaani dataset: {KDV_DATASET_ID} ...")
    try:
        ds = load_dataset(KDV_DATASET_ID, split="train")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return []

    docs = []
    seen = set()
    count = 0

    for i, item in enumerate(ds):
        if limit and count >= limit:
            break

        q = normalize_text(item.get("question", ""))
        a = normalize_text(_resolve_answers_field(item.get("answers", "")))

        if not a or len(a) <= 3 or len(q) <= 3:
            continue

        key = make_key_from_qa(q, a)
        if key in seen:
            continue

        seen.add(key)
        content = f"Question: {q}\nAnswer: {a}"

        docs.append(
            Document(
                page_content=content,
                # 'source' is set to kisanvani so it doesn't get filtered out by PDF rules
                metadata={"source": "kisanvani", "dataset_index": i}
            )
        )
        count += 1

    print(f"Processed {len(docs)} unique Q/A pairs.")
    return docs

# ---------------- LOAD PDF DOCUMENTS ----------------
def load_local_pdfs(path: str) -> List[Document]:
    """Loads all PDFs page-wise, sets filename as 'source'."""
    if not os.path.exists(path) or not os.listdir(path):
        print(f"No PDFs found in {path}. Skipping.")
        return []

    print(f"Loading PDFs from {path} ...")
    loader = PyPDFDirectoryLoader(path)
    pages = loader.load()

    wrapped = []
    seen = set()

    for i, p in enumerate(pages):
        text = normalize_text(getattr(p, "page_content", ""))
        if not text:
            continue

        # Simple dedupe to avoid exact duplicate pages
        key = hashlib.sha1(text[:400].encode()).hexdigest()
        if key in seen:
            continue
        seen.add(key)

        # --- FIX: Extract filename for filtering ---
        full_source_path = p.metadata.get("source", "")
        filename = os.path.basename(full_source_path) # e.g. "pest_control.pdf"

        wrapped.append(
            Document(
                page_content=text,
                metadata={
                    "source": filename, # CRITICAL: Agent uses this for filtering
                    "page": p.metadata.get("page", 0),
                    "full_path": full_source_path
                }
            )
        )

    print(f"Loaded {len(wrapped)} unique PDF pages.")
    return wrapped

# ---------------- BUILD VECTORSTORE ----------------
def build_vectorstore(rebuild: bool = False, use_cuda_if_available: bool = True):
    """Builds and saves FAISS vectorstore."""

    if os.path.exists(VECTORSTORE_PATH) and not rebuild:
        print(f"Vectorstore already exists at {VECTORSTORE_PATH}. Set rebuild=True to overwrite.")
        return

    # Load data
    pdf_docs = load_local_pdfs(PDF_DATA_PATH)
    kdv_docs = load_kisanvani_dataset(limit=MAX_KDV_SAMPLES)

    all_docs = pdf_docs + kdv_docs
    if not all_docs:
        print("No documents found. Add PDFs or check dataset connection.")
        return

    print(f"Total docs before splitting: {len(all_docs)}")

    # Splitting
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    chunks = splitter.split_documents(all_docs)
    print(f"Split into {len(chunks)} chunks.")

    # Dedupe chunks
    unique = {}
    for c in chunks:
        snippet = normalize_text(c.page_content)[:500]
        h = hashlib.sha1(snippet.encode()).hexdigest()
        if h not in unique:
            unique[h] = c

    final_chunks = list(unique.values())
    print(f"{len(final_chunks)} unique chunks after dedupe.")

    # Embeddings
    device = "cuda" if (use_cuda_if_available and torch.cuda.is_available() and USE_CUDA_FOR_EMB) else "cpu"
    print(f"Loading embeddings on {device}...")

    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={"device": device}
    )

    # Build FAISS
    print("Building FAISS index (this may take a while)...")
    vectorstore = FAISS.from_documents(final_chunks, embeddings)
    vectorstore.save_local(VECTORSTORE_PATH)

    print(f"✅ Vectorstore saved successfully at: {VECTORSTORE_PATH}")

# ---------------- QUICK TEST ----------------
def quick_verify_metadata():
    """Checks the first few docs to ensure 'source' is correct."""
    if not os.path.exists(VECTORSTORE_PATH):
        print("No index found to verify.")
        return

    emb = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL, model_kwargs={"device": "cpu"})
    vs = FAISS.load_local(VECTORSTORE_PATH, emb, allow_dangerous_deserialization=True)

    # Peek at internal docstore
    print("\n--- Metadata Verification ---")
    keys = list(vs.docstore._dict.keys())[:5]
    for k in keys:
        doc = vs.docstore.search(k)
        print(f"Source: {doc.metadata.get('source')} | Preview: {doc.page_content[:50]}...")

# ---------------- RUN ----------------
if __name__ == "__main__":
    print("Starting Build Process...")
    # Force rebuild to fix the metadata issue
    build_vectorstore(rebuild=True)

    # Verify the fix
    quick_verify_metadata()

Starting Build Process...
Loading PDFs from /content/drive/MyDrive/Agri_pdfs ...
Loaded 3002 unique PDF pages.
Loading KisanVaani dataset: KisanVaani/agriculture-qa-english-only ...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.97M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22615 [00:00<?, ? examples/s]

Processed 2320 unique Q/A pairs.
Total docs before splitting: 5322
Split into 8823 chunks.
8823 unique chunks after dedupe.
Loading embeddings on cuda...


...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Building FAISS index (this may take a while)...
✅ Vectorstore saved successfully at: faiss_index_bge_m3

--- Metadata Verification ---
Source: Plant-Pathology-and-Disease-Management.pdf | Preview: Plant Pathology and Disease Management (ISBN: 978-...
Source: Plant-Pathology-and-Disease-Management.pdf | Preview: Copyright Editors Title: Plant Pathology and Disea...
Source: Plant-Pathology-and-Disease-Management.pdf | Preview: PREFACE The study of plant pathology and disease m...
Source: Plant-Pathology-and-Disease-Management.pdf | Preview: tools and methods for identifying plant pathogens ...
Source: Plant-Pathology-and-Disease-Management.pdf | Preview: TABLE OF CONTENT Sr. No. Book Chapter and Author(s...


In [5]:
import os
import re
import requests
import torch
from typing import List, TypedDict
from urllib.parse import unquote
import trafilatura
from bs4 import BeautifulSoup
from functools import lru_cache

# LangChain & HuggingFace Imports
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_compressors.flashrank_rerank import FlashrankRerank
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sentence_transformers import SentenceTransformer, util

# --- CONFIG ---
VECTORSTORE_PATH = "faiss_index_bge_m3"
EMBEDDING_MODEL = "BAAI/bge-m3"
DEFAULT_RETRIEVER_K = 10
LLAMA_3_STOP_TAG = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

# --- UTILS ---
def normalize_text(s: str) -> str:
    if not s: return ""
    s = str(s).strip()
    return re.sub(r'\s+', ' ', s)

# --- STATE ---
class GraphState(TypedDict):
    query: str
    queries: List[str]
    documents: List[Document]
    generation: str
    category: str
    verification_passed: bool
    max_sim: float
    web_used: bool

# --- MODEL LOADING (HYBRID: GPU for LLM, CPU for Embeddings) ---

@lru_cache(maxsize=1)
def get_llm(repo_id: str):
    # 1. LLM -> GPU (CUDA)
    print(f"🚀 Loading LLM ({repo_id}) on GPU (CUDA)...")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    tokenizer = AutoTokenizer.from_pretrained(repo_id, token=os.environ.get("HF_TOKEN"))
    model = AutoModelForCausalLM.from_pretrained(
        repo_id,
        quantization_config=bnb_config,
        device_map="auto",
        token=os.environ.get("HF_TOKEN")
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        temperature=0.01,
        eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    )
    return HuggingFacePipeline(pipeline=pipe)

@lru_cache(maxsize=1)
def load_embeddings():
    # 2. EMBEDDINGS -> CPU (To save VRAM for the LLM)
    print(f"⚡ Loading Embeddings on CPU...")
    return HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={'device': 'cpu'}
    )

def get_retriever():
    embeddings = load_embeddings()
    if not os.path.exists(VECTORSTORE_PATH): return None
    vectorstore = FAISS.load_local(VECTORSTORE_PATH, embeddings, allow_dangerous_deserialization=True)
    return vectorstore.as_retriever(search_kwargs={"k": DEFAULT_RETRIEVER_K})

# --- PROMPTS ---
SYSTEM_PROMPT = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are 'AgriBot', an expert agriculturalist.
Answer the user's question based ONLY on the provided context.

**Guidelines:**
1. If the context has the answer, explain it clearly.
2. For pests/diseases: describe symptoms, causes, and controls.
3. For livestock/anatomy: describe the parts/functions clearly based on the text.
4. If the context is empty or irrelevant, say "I don't know".

CONTEXT:
{{context}}<|eot_id|><|start_header_id|>user<|end_header_id|>
{{query}}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
generation_prompt = PromptTemplate(template=SYSTEM_PROMPT, input_variables=["context", "query"])

TRANSFORM_QUERY_TEMPLATE = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Generate 3 search queries for the user's question.
<|eot_id|><|start_header_id|>user<|end_header_id|>
{{query}}{LLAMA_3_STOP_TAG}
"""
TRANSFORM_QUERY_PROMPT = PromptTemplate.from_template(TRANSFORM_QUERY_TEMPLATE)

GRADER_TEMPLATE = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Does the document contain the answer to the question? Respond with 'yes' or 'no'.
<|eot_id|><|start_header_id|>user<|end_header_id|>
DOCUMENTS: {{documents}}
QUESTION: {{query}}{LLAMA_3_STOP_TAG}
"""
GRADER_PROMPT = PromptTemplate(template=GRADER_TEMPLATE, input_variables=["documents", "query"])

# --- NODES ---

def classify_query(state: GraphState) -> GraphState:
    print("---NODE: CLASSIFY QUERY---")
    query = state["query"].lower()

    # 1. KEYWORD OVERRIDE (Livestock)
    livestock_keywords = ["cow", "cattle", "buffalo", "dairy", "goat", "sheep", "poultry", "chicken", "animal", "gut", "stomach", "rumen", "veterinary"]
    if any(k in query for k in livestock_keywords):
        print("Classified via Keywords: livestock")
        return {**state, "category": "livestock"}

    # 2. DISEASE OVERRIDE
    disease_keywords = ["blast", "rust", "rot", "wilt", "mildew", "spot", "blight", "fungus", "fungal", "bacterial", "virus", "viral", "pathogen", "infection", "symptom"]
    if any(k in query for k in disease_keywords):
        print("Classified via Keywords: disease")
        return {**state, "category": "disease"}

    # 3. PEST OVERRIDE
    pest_keywords = ["insect", "worm", "borer", "mite", "beetle", "caterpillar", "larva", "fly", "aphid", "weevil", "bug", "infestation"]
    if any(k in query for k in pest_keywords):
        print("Classified via Keywords: pest")
        return {**state, "category": "pest"}

    # 4. LLM FALLBACK
    try:
        llm = get_llm(repo_id="meta-llama/Llama-3.1-8B-Instruct")
        prompt = f"Classify into: pest, disease, soil, water, machinery, livestock, ipm.\nQuery: {query}\nCategory:"
        cat = llm.invoke(prompt).strip().lower()
        valid = ["pest", "disease", "soil", "water", "machinery", "ipm", "livestock"]
        category = next((c for c in valid if c in cat), "general")
    except:
        category = "general"

    print(f"Classified via LLM as: {category}")
    return {**state, "category": category}

def generate_queries(state: GraphState) -> GraphState:
    print("---NODE: GENERATE QUERIES---")
    try:
        llm = get_llm(repo_id="meta-llama/Llama-3.1-8B-Instruct")
        chain = TRANSFORM_QUERY_PROMPT | llm | StrOutputParser()
        res = chain.invoke({"query": state["query"]})
        if LLAMA_3_STOP_TAG in res: res = res.split(LLAMA_3_STOP_TAG)[-1]
        qs = [q.strip() for q in res.split('\n') if q.strip()]
        return {**state, "queries": [state["query"]] + qs}
    except:
        return {**state, "queries": [state["query"]]}

# --- RETRIEVAL ---
PDF_CLUSTERS = {
    "pest": ["pest", "insect", "management"],
    "disease": ["disease", "pathology", "fungus", "bacterial", "viral", "blast", "rust"],
    "soil": ["soil", "manure", "fertilizer", "nutrient"],
    "water": ["water", "irrigation"],
    "machinery": ["machinery", "tractor", "farm-power"],
    "livestock": ["livestock", "dairy", "cow", "cattle", "animal", "production"]
}

def retrieve_local(state: GraphState) -> GraphState:
    print("---NODE: RETRIEVE LOCAL---")
    retriever = get_retriever()
    if not retriever: return state

    docs = []
    # Use only first query to save time
    search_queries = state.get("queries", [state["query"]])
    try: docs.extend(retriever.invoke(search_queries[0]))
    except: pass

    # Soft Filter
    category = state.get("category", "general")
    allowed = PDF_CLUSTERS.get(category, [])
    if allowed:
        filtered = [d for d in docs if any(k.lower() in str(d.metadata).lower() for k in allowed)]
        # Fallback to ALL docs if filter removes everything
        if filtered: docs = filtered
        else: print(f"Filter '{category}' returned 0. Using all docs.")

    return {**state, "documents": docs[:10]} # Limit quantity

def rerank_documents(state: GraphState) -> GraphState:
    print("---NODE: RERANK---")
    docs = state["documents"]
    if not docs: return state
    ranker = FlashrankRerank(top_n=5)
    return {**state, "documents": ranker.compress_documents(documents=docs, query=state["query"])}

def grade_local_docs(state: GraphState) -> GraphState:
    print("---NODE: GRADE---")
    docs = state["documents"]
    if not docs: return {**state, "generation": "not_relevant"}

    try:
        llm = get_llm(repo_id="meta-llama/Llama-3.1-8B-Instruct")
        txt = "\n".join([d.page_content for d in docs])
        res = (GRADER_PROMPT | llm | StrOutputParser()).invoke({"documents": txt, "query": state["query"]})
        if "yes" in res.lower():
            print("Graded: RELEVANT")
            return {**state, "generation": "use_local"}
    except: pass
    print("Graded: NOT RELEVANT")
    return {**state, "generation": "not_relevant"}

# --- WEB SEARCH ---
def web_search_ddg(query):
    try:
        url = f"https://duckduckgo.com/html/?q={requests.utils.requote_uri(query)}"
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, "html.parser")
        links = []
        for l in soup.select(".result__a")[:3]: # Limit to Top 3
            raw = l.get("href")
            if "uddg=" in raw:
                try: links.append(unquote(raw.split("uddg=")[1].split("&")[0]))
                except: pass
            else: links.append(raw)
        return links
    except: return []
def scrape_page(url):
    try:
        if url.startswith("//"): url = "https:" + url
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=5)
        if r.status_code == 200:
            text = trafilatura.extract(r.text)
            # LIMIT CONTEXT to 2000 characters
            return text[:2000] if text else ""
    except: pass
    return ""
def web_rag(state: GraphState) -> GraphState:
    print("---NODE: WEB SEARCH---")
    links = web_search_ddg(state["query"])
    docs = []
    for link in links:
        txt = scrape_page(link)
        if txt: docs.append(Document(page_content=normalize_text(txt), metadata={"source": link}))

    if not docs: return {**state, "generation": "use_web", "web_used": True, "documents": []}
    print(f"Web retrieved {len(docs)} pages.")
    return {**state, "documents": docs, "generation": "use_web", "web_used": True}

# --- GENERATION ---
def generate(state: GraphState, llm) -> GraphState:
    print("---NODE: GENERATE---")
    docs = state["documents"]
    if not docs: return {**state, "generation": "I'm sorry, I couldn't find any information."}

    context = "\n\n".join([d.page_content for d in docs])
    try:
        res = (generation_prompt | llm | StrOutputParser()).invoke({"context": context, "query": state["query"]})
        if LLAMA_3_STOP_TAG in res: res = res.split(LLAMA_3_STOP_TAG)[-1]
        return {**state, "generation": res}
    except Exception as e:
        return {**state, "generation": f"Error: {e}"}

def verify_generation(state: GraphState) -> GraphState:
    print("---NODE: VERIFY---")
    # 1. Skip if Web was used
    if state.get("web_used"):
        print("Web source used: Skipping strict verification.")
        return {**state, "verification_passed": True}

    gen = state.get("generation", "")
    docs = state.get("documents", [])
    if not gen or not docs: return {**state, "verification_passed": False}

    # 2. Similarity Check (Uses CPU Embeddings)
    try:
        emb = load_embeddings()
        v_gen = emb.embed_query(gen)
        v_docs = emb.embed_documents([d.page_content for d in docs])
        sims = util.cos_sim(torch.tensor(v_gen), torch.tensor(v_docs))
        if float(sims.max()) > 0.60:
            print("Verified via Similarity.")
            return {**state, "verification_passed": True}
    except: pass

    print("Verification Failed.")
    return {**state, "verification_passed": False}

# --- EDGES & GRAPH ---
def decide_retrieval_path(state):
    return "generate" if state.get("generation") == "use_local" else "web_rag_node"

def check_hallucination(state):
    if state.get("verification_passed"): return "useful"
    if not state.get("web_used"): return "not_supported"
    return "useful"

def build_graph(llm):
    workflow = StateGraph(GraphState)
    workflow.add_node("classify_query", classify_query)
    workflow.add_node("generate_queries", generate_queries)
    workflow.add_node("retrieve_local", retrieve_local)
    workflow.add_node("rerank_documents", rerank_documents)
    workflow.add_node("grade_local_docs", grade_local_docs)
    workflow.add_node("web_rag_node", web_rag)
    workflow.add_node("generate", lambda s: generate(s, llm))
    workflow.add_node("verify_generation", verify_generation)

    workflow.set_entry_point("classify_query")
    workflow.add_edge("classify_query", "generate_queries")
    workflow.add_edge("generate_queries", "retrieve_local")
    workflow.add_edge("retrieve_local", "rerank_documents")
    workflow.add_edge("rerank_documents", "grade_local_docs")

    workflow.add_conditional_edges("grade_local_docs", decide_retrieval_path, {"generate": "generate", "web_rag_node": "web_rag_node"})
    workflow.add_edge("web_rag_node", "generate")
    workflow.add_edge("generate", "verify_generation")

    workflow.add_conditional_edges("verify_generation", check_hallucination, {"useful": END, "not_supported": "web_rag_node"})
    return workflow.compile()



print("AgriBot v5 (Final Robust) loaded.")

AgriBot v5 (Final Robust) loaded.


In [6]:
# --- INITIALIZATION ---
print("Initializing the AgriBot...")
try:
    # Load LLM on GPU, Embeddings on CPU
    main_llm = get_llm(repo_id="meta-llama/Llama-3.1-8B-Instruct")
    app = build_graph(main_llm)
    print("✅ AgriBot is ready! Ask your question below.")
except Exception as e:
    print(f"❌ Failed to initialize agent. Error: {repr(e)}")

# --- CHAT LOOP ---
if 'app' in locals():
    while True:
        query = input("\n[You]: ")
        if query.lower() in ['exit', 'quit']:
            break

        inputs = {"query": query}
        print("\n[AgriBot is thinking...]")

        result = app.invoke(inputs, config={"recursion_limit": 50})
        final_answer = result.get("generation", "I am unable to find an answer.")
        source_tag = "[Source: Web Search 🌐]" if result.get("web_used") else "[Source: Local Manuals 📚]"

        print(f"\n[🧑‍🌾 AgriBot] {source_tag}:\n{final_answer}")

Initializing the AgriBot...
🚀 Loading LLM (meta-llama/Llama-3.1-8B-Instruct) on GPU (CUDA)...


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

Classified via LLM as: pest
---NODE: GENERATE QUERIES---
---NODE: RETRIEVE LOCAL---
---NODE: RERANK---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


---NODE: GRADE---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Graded: RELEVANT
---NODE: GENERATE---
---NODE: VERIFY---
Verified via Similarity.

[🧑‍🌾 AgriBot] [Source: Local Manuals 📚]:

Based on the provided context, the white powder on bhendi leaves appears to be powdery mildew, caused by the fungus Erysiphe polygonii. The symptoms described match the ones mentioned in the text, such as white powdery patches on the leaves, which can lead to the entire leaf surface being covered with a powdery coating.

To control powdery mildew, the recommended management is to spray Wettable sulphur 0.25% or Dinocap (Karathane) 0.05%.

[You]: exit


In [7]:
def diagnose_index():
    print("--- DIAGNOSING VECTOR STORE ---")
    try:
        # Load the index
        embeddings = load_embeddings()
        vectorstore = FAISS.load_local(VECTORSTORE_PATH, embeddings, allow_dangerous_deserialization=True)

        # Access the internal docstore to list sources
        docstore_dict = vectorstore.docstore._dict
        unique_sources = set()

        for doc_id, doc in docstore_dict.items():
            if "source" in doc.metadata:
                unique_sources.add(doc.metadata["source"])

        print(f"✅ Found {len(unique_sources)} unique documents in the index:")
        for src in unique_sources:
            print(f" - {src}")

        return list(unique_sources)
    except Exception as e:
        print(f"❌ Error reading index: {e}")
        return []

# Run the diagnosis
sources = diagnose_index()

--- DIAGNOSING VECTOR STORE ---
✅ Found 17 unique documents in the index:
 - Livestock-Production-and-Management.pdf
 - INTRODUCTION TO SOIL SCIENCE.pdf
 - MANURES, FERTILIZERS AND AGROCHEMICALS.pdf
 - Weed-Management.pdf
 - Manures-Fertilizers-Agrochemicals.pdf
 - Plant-Pathogens-Principles-of-Plant-Pathology.pdf
 - Diseases-of-Field-Crops-and-Their-Management.pdf
 - Crop-Pests-and-Stored-Grain-Pests-and-Their-Management.pdf
 - Soil_Science.pdf
 - PRINCIPLES OF AGRONOMY AND AGRICULTURAL METEOROLOGY.pdf
 - Disease-of-Horticultural-Crops-their-Management1.pdf
 - kisanvani
 - LIVESTOCK PRODUCTION AND MANAGEMENT.pdf
 - Plant-Pathology-and-Disease-Management.pdf
 - INSECT ECOLOGY & INTEGRATED PEST MANAGEMENT INCLUDING BENEFICIAL INSECTS.pdf
 - Water-Management-including-Micro-Irrigation.pdf
 - FARM-POWER-AND-MACHINERY.pdf


In [ ]:
# ---------- Generate answers and save to CSV ----------
import pandas as pd
import ast
import time

# Path settings
INPUT_EVAL_CSV = "/content/drive/MyDrive/evaluation_English.csv"   # source questions
OUTPUT_PRED_CSV = "/content/drive/MyDrive/evaluation_predictions.csv"  # generated answers saved here

print("Loading evaluation CSV:", INPUT_EVAL_CSV)

try:
    eval_df = pd.read_csv(INPUT_EVAL_CSV)
except FileNotFoundError:
    raise FileNotFoundError(f"Evaluation CSV not found at {INPUT_EVAL_CSV}. Upload it to Drive and re-run this cell.")

# Basic column checks
if "Question" not in eval_df.columns or "Ground Truth Answer" not in eval_df.columns:
    raise ValueError("CSV must contain columns 'Question' and 'Ground Truth Answer'.")

questions = eval_df["Question"].tolist()
ground_truths = eval_df["Ground Truth Answer"].tolist()

# Prepare agent
print("Initializing LLM and agent graph (this will reuse your existing get_llm and build_graph functions)...")
llm_for_eval = get_llm(repo_id="meta-llama/Llama-3.1-8B-Instruct")
app_for_eval = build_graph(llm_for_eval)

answers = []
contexts = []

print(f"Running RAG agent for {len(questions)} questions. This may take a while depending on model & infra.")
start_time = time.time()
for i, q in enumerate(questions, start=1):
    print(f"[{i}/{len(questions)}] Query:", q)
    try:
        # keep recursion_limit as before (if your graph uses it)
        state = app_for_eval.invoke({"query": q}, config={"recursion_limit": 10})
        ans = state.get("generation", "") or ""
        docs = state.get("documents", []) or []
        ctxs = [d.page_content for d in docs]
    except Exception as e:
        print(f"  ⚠ Error on query {i}: {repr(e)} — storing empty answer/context")
        ans = ""
        ctxs = []

    answers.append(ans)
    contexts.append(ctxs)

elapsed = time.time() - start_time
print(f"Finished generation for {len(questions)} queries in {elapsed:.1f}s.")

# Save to DataFrame and CSV
out_df = pd.DataFrame({
    "question": questions,
    "ground_truth": ground_truths,
    "answer": answers,
    "contexts": [repr(c) for c in contexts]
})

out_df.to_csv(OUTPUT_PRED_CSV, index=False)
print(f"Saved predictions and contexts to: {OUTPUT_PRED_CSV}")
print("Sample rows:")
print(out_df.head(5))


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Loading evaluation CSV: /content/drive/MyDrive/evaluation_English.csv
Initializing LLM and agent graph (this will reuse your existing get_llm and build_graph functions)...
Running RAG agent for 25 questions. This may take a while depending on model & infra.
[1/25] Query: What is crop rotation?
---NODE: GENERATE QUERIES---
Original query: 'What is crop rotation?'
Generated queries: ['What is crop rotation?', 'Crop rotation in permaculture', 'Benefits of crop rotation in agriculture', 'Crop rotation techniques for small farms']
---NODE: RETRIEVE LOCAL---
Loading FAISS vector store...
Retrieved 16 unique documents locally.
---NODE: RERANK DOCUMENTS---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Reranked 16 docs down to 6.
---NODE: GRADE LOCAL DOCS---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Max semantic similarity (cached embeddings) between generation and contexts: 0.635
Generation NOT supported by documents. Replacing with fallback message.
[25/25] Query: What is the role of potassium in plants?
---NODE: GENERATE QUERIES---
Original query: 'What is the role of potassium in plants?'
Generated queries: ['What is the role of potassium in plants?', 'What are the symptoms of potassium deficiency in plants?', 'How does potassium affect plant growth and development?', 'What are the benefits of potassium fertilizers for crop yields?']
---NODE: RETRIEVE LOCAL---
Loading FAISS vector store...
Retrieved 27 unique documents locally.
---NODE: RERANK DOCUMENTS---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Reranked 27 docs down to 6.
---NODE: GRADE LOCAL DOCS---


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Grader raw output: 'yes'
Decision: Local documents are RELEVANT.
---NODE: GENERATE---
---NODE: VERIFY GENERATION (CACHED EMBEDDINGS)---
Max semantic similarity (cached embeddings) between generation and contexts: 0.755
Generation is semantically supported by retrieved documents.
Finished generation for 25 queries in 1139.2s.
Saved predictions and contexts to: /content/drive/MyDrive/evaluation_predictions.csv
Sample rows:
                                            question  \
0                             What is crop rotation?   
1  What are the main symptoms of rice blast disease?   
2  What is the difference between macronutrients ...   
3    How does integrated pest management (IPM) work?   
4        What is the purpose of adding lime to soil?   

                                        ground_truth  \
0  Crop rotation is the practice of growing a ser...   
1  Symptoms include spindle-shaped lesions on lea...   
2  Macronutrients (like N, P, K) are needed by pl...   
3  IPM is an e

In [ ]:
# ---------- Load predictions CSV and compute metrics ----------
import pandas as pd
import ast
import warnings
import numpy as np
from sentence_transformers import SentenceTransformer, util
import evaluate as evaluate_lib

warnings.filterwarnings("ignore")

# Paths (must match Cell 1)
PRED_CSV = "/content/drive/MyDrive/evaluation_predictions.csv"
EVAL_EMBEDDING_MODEL = "BAAI/bge-m3"  # used for cosine similarity

# load predictions
print("Loading predictions from:", PRED_CSV)
pred_df = pd.read_csv(PRED_CSV)

# parse contexts column (saved as repr(list))
def parse_contexts(cell):
    if pd.isna(cell):
        return []
    try:
        parsed = ast.literal_eval(cell)
        if isinstance(parsed, list):
            return parsed
        else:
            return [str(parsed)]
    except Exception:
        # final fallback: treat as single string
        return [str(cell)]

pred_df["contexts"] = pred_df["contexts"].apply(parse_contexts)

# Load metrics
print("Loading metrics (BLEU, ROUGE, BERTScore) and embedding model (cpu)...")
bleu_metric = evaluate_lib.load("bleu")
rouge_metric = evaluate_lib.load("rouge")
bertscore_metric = evaluate_lib.load("bertscore")
similarity_model = SentenceTransformer(EVAL_EMBEDDING_MODEL, device="cpu")

# recall@k helper (same as you used)
def compute_recall_at_k(ground_truth, contexts, similarity_model, k_list=(1,3,5), sim_threshold=0.65):
    results = {f"recall@{k}": 0 for k in k_list}
    if contexts is None or len(contexts) == 0:
        return results
    gt_lower = str(ground_truth).casefold()
    relevance_flags = [1 if (ctx is not None and gt_lower in str(ctx).casefold()) else 0 for ctx in contexts]
    if any(relevance_flags):
        matched_idx = [i for i, v in enumerate(relevance_flags) if v == 1]
        for k in k_list:
            results[f"recall@{k}"] = 1 if any(idx < k for idx in matched_idx) else 0
        return results
    safe_contexts = ["" if ctx is None else str(ctx) for ctx in contexts]
    try:
        gt_emb = similarity_model.encode(ground_truth, convert_to_tensor=True)
        ctx_emb = similarity_model.encode(safe_contexts, convert_to_tensor=True)
        sims = util.cos_sim(gt_emb, ctx_emb).cpu().numpy().reshape(-1)
    except Exception:
        return results
    sorted_idx = list(np.argsort(-sims))
    for k in k_list:
        top_k = sorted_idx[:k]
        if any(sims[i] >= sim_threshold for i in top_k):
            results[f"recall@{k}"] = 1
    return results

# Initialize metric lists
bleu_scores = []
rouge_l_scores = []
cosine_scores = []
bert_f1_scores = []
recall_at_1 = []
recall_at_3 = []
recall_at_5 = []

print("Computing metrics for each example...")
for _, row in pred_df.iterrows():
    prediction = str(row.get("answer","") or "")
    reference = str(row.get("ground_truth","") or "")
    contexts = row.get("contexts", []) or []

    if not prediction or not reference or "Error" in prediction:
        bleu_scores.append(0.0)
        rouge_l_scores.append(0.0)
        cosine_scores.append(0.0)
        bert_f1_scores.append(0.0)
        recall_at_1.append(0)
        recall_at_3.append(0)
        recall_at_5.append(0)
        continue

    # BLEU
    try:
        bleu_res = bleu_metric.compute(predictions=[prediction], references=[reference])
        bleu_scores.append(bleu_res.get("bleu", 0.0))
    except Exception:
        bleu_scores.append(0.0)

    # ROUGE-L
    try:
        rouge_res = rouge_metric.compute(predictions=[prediction], references=[reference])
        rL = rouge_res.get("rougeL", 0.0)
        rL = rL["fmeasure"] if isinstance(rL, dict) and "fmeasure" in rL else rL
        rouge_l_scores.append(float(rL or 0.0))
    except Exception:
        rouge_l_scores.append(0.0)

    # Cosine similarity
    try:
        e1 = similarity_model.encode(prediction, convert_to_tensor=True)
        e2 = similarity_model.encode(reference, convert_to_tensor=True)
        cosine_scores.append(util.cos_sim(e1, e2).item())
    except Exception:
        cosine_scores.append(0.0)

    # BERTScore
    try:
        bert = bertscore_metric.compute(predictions=[prediction], references=[reference], lang="en")
        bert_f1_scores.append(np.mean(bert["f1"]))
    except Exception:
        bert_f1_scores.append(0.0)

    # Recall@K
    r = compute_recall_at_k(reference, contexts, similarity_model)
    recall_at_1.append(r["recall@1"])
    recall_at_3.append(r["recall@3"])
    recall_at_5.append(r["recall@5"])

# Append to dataframe
pred_df["bleu_score"] = bleu_scores
pred_df["rougeL_score"] = rouge_l_scores
pred_df["cosine_similarity"] = cosine_scores
pred_df["bertscore_f1"] = bert_f1_scores
pred_df["recall@1"] = recall_at_1
pred_df["recall@3"] = recall_at_3
pred_df["recall@5"] = recall_at_5

# Final aggregated report
print("\n--- FINAL EVALUATION REPORT ---\n")
print(f"Average BLEU:         {np.mean(bleu_scores):.4f}")
print(f"Average ROUGE-L:      {np.mean(rouge_l_scores):.4f}")
print(f"Average Cosine Sim:   {np.mean(cosine_scores):.4f}")
print(f"Average BERTScore F1: {np.mean(bert_f1_scores):.4f}")
print(f"Average Recall@1:     {np.mean(recall_at_1):.4f}")
print(f"Average Recall@3:     {np.mean(recall_at_3):.4f}")
print(f"Average Recall@5:     {np.mean(recall_at_5):.4f}")

# Save detailed results (optional)
OUT_DETAIL = "/content/drive/MyDrive/evaluation_results_with_all_metrics.csv"
pred_df.to_csv(OUT_DETAIL, index=False)
print(f"\nSaved detailed results to: {OUT_DETAIL}")

# Print small sample
print("\nSample rows:")
print(pred_df.head(5))


Loading predictions from: /content/drive/MyDrive/evaluation_predictions.csv
Loading metrics (BLEU, ROUGE, BERTScore) and embedding model (cpu)...
Computing metrics for each example...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- FINAL EVALUATION REPORT ---

Average BLEU:         0.0312
Average ROUGE-L:      0.2064
Average Cosine Sim:   0.8097
Average BERTScore F1: 0.8663
Average Recall@1:     0.9200
Average Recall@3:     0.9200
Average Recall@5:     0.9200

Saved detailed results to: /content/drive/MyDrive/evaluation_results_with_all_metrics.csv

Sample rows:
                                            question  \
0                             What is crop rotation?   
1  What are the main symptoms of rice blast disease?   
2  What is the difference between macronutrients ...   
3    How does integrated pest management (IPM) work?   
4        What is the purpose of adding lime to soil?   

                                        ground_truth  \
0  Crop rotation is the practice of growing a ser...   
1  Symptoms include spindle-shaped lesions on lea...   
2  Macronutrients (like N, P, K) are needed by pl...   
3  IPM is an ecosystem-based strategy that focuse...   
4  Adding lime to soil is primarily done 